<a href="https://colab.research.google.com/github/jcdevaney/pyAMPACTtutorials/blob/main/05-pyAMPACT_symbolic_Importing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>pyAMPACT Importing Symbolic Data</h1>



In [1]:
from IPython.utils import io
print('Importing libraries...')
with io.capture_output() as captured:
    !pip install --upgrade pandas
    !pip install -i https://test.pypi.org/simple/ --no-deps pyampact
    import pyampact
    !pip install crim_intervals
    import crim_intervals as crim

Importing libraries...


In [2]:
piece_url = 'https://github.com/pyampact/pyAMPACTtutorials/blob/main/test_files/O_Virgo_Miserere.krn'
pyamp_piece = pyampact.Score(piece_url)
rdiss = pyamp_piece.getSpines('cdata-rdiss')
rdiss


	Detected and imported these spine types:
		 cdata-rdiss 



This notebook demonstrates how to use pyAMPACT with Humdrum. By "Humdrum" we are referring to the whole Humdrum ecosystem of analysis tools including humlib and the Verovio Humdrum Viewer. On its own, pyAMPACT already supports .krn as a symbolic notation file type for both importing and exporting/converting files. So this notebook goes beyond that and explains how to integrate Humdrum analysis tools into a pyAMPACT workflow.

As a practical example, we get Renaissance dissonance analysis from the humlib dissonant tool. We then use pyAMPACT see which of the dissonance types correlate with various observed audio phenomena in a recording of the same piece. Since the two main components of this workflow are distributed in the Humdrum and pyAMPACT toolkits, the only way to realize this workflow is by combining these otherwise separate tools.

Now let's get the score and analysis data. The critical thing here is that **the Humdrum analysis must be in the same .krn file as the score itself**. So in addition to having as many **kern spines as there are voices in the piece, the krn file should have at least one spine of analysis data. As a reference, there are several special spine types that we have dedicated methods for reading in as pandas dataframes in pyAMPACT. This table shows which pyAMPACT Score object methods to use to get each given .krn file spine type:

| Spine Type | pyAMPACT Methods |
|----------|----------|
| **kern | .notes(), .midiPitches() |
| **text | .lyrics() |
| **dyanm | .dynamics() |
| **harm | .harm(), .harmKeys(), .romanNumeral()|
| **function | .functions() |
| **chord | .chords() |
| **cdata | .cdata() |
| Any other spine type | .getSpines('name_of_spine_type') |

In this notebook we're using .getSpines() to get `**cdata-rdiss` spine data into a score-time aligned pandas dataframe. We're using .getSpines because it's the generic spine importer pyAMPACT exposes for any spine types not explicitly listed in the table above.

Humdrum's Renaissance dissonance classification tool is part of humlib and is also accessible on the Verovio Humdrum Viewer. You can go to any Renaissance score, then type "dissonant" in the text box and hit return. Then press the `+` button to the right of the text box to compile the filter which will put the analysis labels in their own spines in the .krn file. This is what that looks like for Obrecht's [Kyrie, Missa Ave regina celorum](https://verovio.humdrum.org/?file=jrp/Obr/Obr1002a-Missa_Ave_regina_celorum-Kyrie.krn).

Alternatively you can add `!!!filter: dissonant` to the top of the .krn file on the left side of the score and then press`alt + c` to compile the filter (`option + c` on a mac). Then you can copy the full text and save it as your .krn file. Still another option is to use Humdrum from the terminal to create your score with analysis.

In this tutorial, we've already combined that Renaissance dissonance analysis with a Humdrum file of a Tinctoris score, available [here](https://github.com/pyampact/pyAMPACTtutorials/blob/main/test_files/O_Virgo_Miserere.krn). We've already imported that piece with pyAMPACT and examined a table of the dissonance results, so now let's use CRIM to get a different type of analysis of the same piece.

<h2>Use CRIM analysis in a pyAMPACT workflow<h2>

In [5]:
from IPython.utils import io
print('Importing CRIM...')
with io.capture_output() as captured:
    !pip install crim_intervals
    import crim_intervals as crim
piece_url = 'https://github.com/pyampact/pyAMPACTtutorials/blob/main/test_files/O_Virgo_Miserere.krn'
crim_piece = crim.importScore(piece_url)


	Detected and imported these spine types:
		 cdata-rdiss 

Import of  failed, please check your file, path, or url.


We'll use CRIM's Cadential Voice Function analysis, .cvfs() and then convert this piece into an MEI file that contains these CVF annotations using pyAMPACT.

CRIM's CVF dataframe can be treated as if it's a pyAMPACT dataframe without any special treatment because the dataframe's index is time-aligned and the columnar index consists of part names.

In a couple of cases, CRIM dataframes differ from this structure (for example .cadences() has different columns and .presentationTypes() has a different index and columns). You can still work with this CRIM data in pyAMPACT in these cases, they just require some basic restructuring of the CRIM dataframes first.

Once you run the next cell, an MEI file call "pyAMPACT-CRIM_test.mei.xml" will appear in the menu on the left at `/pyAMPACT-CRIM_test.mei.xml`. It will be too big to view in colab, but if you double-click it you can download it.

In [6]:
cvf_table = crim_piece.cvfs()
pyamp_piece.toMEI('pyAMPACT-CRIM_test', ' ', dfs={'CVF': cvf_table}, analysis_tag='harm')

AttributeError: 'NoneType' object has no attribute 'cvfs'

pyAMPACT can also create links to short excerpts of pieces, with or without extra data. We do this in the .krn format for its character economy and extensibility, via the (Verovio Humdrum Viewer)[https://verovio.humdrum.org]. Let's look at the cadential voice functions used in the last cadence of this piece which is in the last two measures. Soon we'll have the ability to include annotations in .krn output too, and then these will optionally include the annotations. But for now, let's just look at the last two measures.

In [7]:
pyamp_piece.show(start=45)

https://verovio.humdrum.org/?t=ISEhQ09NOiBUaW5jdG9yaXMsIEpvaGFubmVzCiEhIU9UTDogTyB2aXJnbyBtaXNlcmVyZSBtZWkKKiprZXJuCSoqa2VybgkqKmtlcm4KKnBhcnQzCSpwYXJ0MgkqcGFydDEKKnN0YWZmMwkqc3RhZmYyCSpzdGFmZjEKKkl2b3gJKkl2b3gJKkl2b3gKKkkiUGFydC0zCSpJIlBhcnQtMgkqSSJWb2ljZQoqSSdQCSpJJ1AJKkknVgo9NDUJPTQ1CT00NQoxQUFdCTRlXQk0Y2NdCi4JNGQJNGIKLgkyYwkxYQoxRQkxQgkuCi4JLgkyZyMKPTQ2CT00Ngk9NDYKMEFBCTBBCTBhCj09CT09CT09CiotCSotCSotCiEhIVJERioqa2VybjogJT1yYXRpb25hbCByaHl0aG0KISEhUkRGKiprZXJuOiBsPWxvbmcgbm90ZSBpbiBvcmlnaW5hbCBub3RhdGlvbgohISFSREYqKmtlcm46IGk9ZWRpdG9yaWFsIGFjY2lkZW50YWwKISEhT05COiBUcmFuc2xhdGVkIGZyb20gYSBrcm4gZmlsZSBvbiAyMDI0LTAzLTEzIHZpYSBweUFNUEFDVAohISF0aXRsZTogQHtPVEx9
